# Add Derived Graphs To The Tutorial Graph



In [1]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd
from IPython.display import display, HTML

import papermill as pm

sys.path.insert(0,'../..')
from configure_kgtk_notebooks import ConfigureKGTK

from kgtk.functions import kgtk, kypher

In [2]:
# Parameters

kgtk_path = "/Users/pedroszekely/Documents/GitHub/kgtk"

# Folder on local machine where to create the output and temporary folders
input_path = "/Users/pedroszekely/Downloads/kypher/projects/build-tutorial"
output_path = "/Users/pedroszekely/Downloads/kypher/projects"
project_name = "tutorial-derived-graphs"
tutorial_files_path = "/Users/pedroszekely/Documents/GitHub/kgtk-tutorial-files/datasets/arnold"

In [3]:
files = [
    "all"
]
ck = ConfigureKGTK(kgtk_path=kgtk_path)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name)

User home: /Users/pedroszekely
Current dir: /Users/pedroszekely/Documents/GitHub/kgtk/tutorial/build-kg
KGTK dir: /Users/pedroszekely/Documents/GitHub/kgtk
Use-cases dir: /Users/pedroszekely/Documents/GitHub/kgtk/use-cases


In [4]:
ck.print_env_variables(files)

GRAPH: /Users/pedroszekely/Downloads/kypher/projects/build-tutorial
kypher: kgtk query --graph-cache /Users/pedroszekely/Downloads/kypher/projects/tutorial-derived-graphs/temp.tutorial-derived-graphs/wikidata.sqlite3.db
USE_CASES_DIR: /Users/pedroszekely/Documents/GitHub/kgtk/use-cases
TEMP: /Users/pedroszekely/Downloads/kypher/projects/tutorial-derived-graphs/temp.tutorial-derived-graphs
STORE: /Users/pedroszekely/Downloads/kypher/projects/tutorial-derived-graphs/temp.tutorial-derived-graphs/wikidata.sqlite3.db
kgtk: kgtk
OUT: /Users/pedroszekely/Downloads/kypher/projects/tutorial-derived-graphs
EXAMPLES_DIR: /Users/pedroszekely/Documents/GitHub/kgtk/examples
all: /Users/pedroszekely/Downloads/kypher/projects/build-tutorial/all.tsv.gz


Turn on debugging for kypher

In [5]:
os.environ['tutorial_files_path'] = tutorial_files_path
os.environ['kgtk_path'] = kgtk_path
os.environ['KGTK_GRAPH_CACHE'] = os.environ['STORE']
os.environ['KGTK_LABEL_FILE'] = os.environ['OUT'] + "/parts/labels.en.tsv.gz"
os.environ['KGTK_OPTION_DEBUG'] = "true"

Load all my files into the kypher cache so that all graph aliases are defined

In [6]:
ck.load_files_into_cache(file_list=files)

kgtk query --graph-cache /Users/pedroszekely/Downloads/kypher/projects/tutorial-derived-graphs/temp.tutorial-derived-graphs/wikidata.sqlite3.db -i "/Users/pedroszekely/Downloads/kypher/projects/build-tutorial/all.tsv.gz" --as all  --limit 3
[2021-10-08 17:12:05 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [3]
---------------------------------------------
node1	label	node2	id
P10	P31	Q18610173	P10-P31-Q18610173-85ef4d24-0
P1000	P31	Q18608871	P1000-P31-Q18608871-093affb5-0
P1001	P1647	P276	P1001-P1647-P276-e4e44f83-0


In [7]:
%cd {os.environ['OUT']}

/Users/pedroszekely/Downloads/kypher/projects/tutorial-derived-graphs


## Run partition notebook

We need the parts to run the Useful Files notebook

In [8]:
pm.execute_notebook(
    os.environ["EXAMPLES_DIR"] + "/partition-wikidata.ipynb",
    os.environ["TEMP"] + "/partition-wikidata.out.ipynb",
    parameters=dict(
        wikidata_input_path = input_path + "/all.tsv.gz",
        wikidata_parts_path = os.environ["OUT"] + "/parts",
        temp_folder_path = os.environ["OUT"] + "/parts/temp",
        sort_extras = "--buffer-size 30% --temporary-directory $OUT/parts/temp",
        verbose = False,
        gzip_command = 'gzip'
    )
)
;

Executing:   0%|          | 0/49 [00:00<?, ?cell/s]

''

Show the files after partition

In [9]:
!ls $OUT/parts

aliases.en.tsv.gz                   metadata.property.datatypes.tsv.gz
aliases.tsv.gz                      metadata.types.tsv.gz
all.tsv.gz                          qualifiers.commonsMedia.tsv.gz
claims.commonsMedia.tsv.gz          qualifiers.external-id.tsv.gz
claims.external-id.tsv.gz           qualifiers.geo-shape.tsv.gz
claims.geo-shape.tsv.gz             qualifiers.globe-coordinate.tsv.gz
claims.globe-coordinate.tsv.gz      qualifiers.math.tsv.gz
claims.math.tsv.gz                  qualifiers.monolingualtext.tsv.gz
claims.monolingualtext.tsv.gz       qualifiers.musical-notation.tsv.gz
claims.musical-notation.tsv.gz      qualifiers.quantity.tsv.gz
claims.other.tsv.gz                 qualifiers.string.tsv.gz
claims.quantity.tsv.gz              qualifiers.tabular-data.tsv.gz
claims.string.tsv.gz                qualifiers.time.tsv.gz
claims.tabular-data.tsv.gz          qualifiers.tsv.gz
claims.time.tsv.gz                  qualifiers.url.tsv.gz
claims.tsv.gz                       quali

Deploy the parts to `$tutorial_files_path`

## Run useful files notebook

In [10]:
pm.execute_notebook(
    os.environ["USE_CASES_DIR"] + "/Wikidata Useful Files.ipynb",
    os.environ["TEMP"] + "/Wikidata Useful Files Out.ipynb",
    parameters=dict(
        output_path = os.environ["OUT"],
        output_folder = "useful_files",
        temp_folder = "temp.useful_files",
        wiki_root_folder = os.environ["OUT"] + "/parts/",
        cache_path = os.environ["TEMP"],
        languages = 'en',
        compute_pagerank = True,
        compute_degrees = True,
        compute_hits = False, 
        delete_database = False,
        debug = "false"
    )
)
;

Executing:   0%|          | 0/157 [00:00<?, ?cell/s]

''

In [11]:
!ls -l $OUT/useful_files

total 46304
-rw-r--r--   1 pedroszekely  staff  1167663 Oct  8 13:41 aliases.en.tsv.gz
-rw-r--r--   1 pedroszekely  staff   326969 Oct  8 13:42 derived.P279.tsv.gz
-rw-r--r--   1 pedroszekely  staff  2649699 Oct  8 13:42 derived.P279star.tsv.gz
-rw-r--r--   1 pedroszekely  staff  1006728 Oct  8 13:41 derived.P31.tsv.gz
-rw-r--r--   1 pedroszekely  staff       47 Oct  8 13:43 derived.dwd.count.tsv.gz
-rw-r--r--   1 pedroszekely  staff  1540685 Oct  8 13:43 derived.dwd_isa.tsv.gz
-rw-r--r--   1 pedroszekely  staff  1158470 Oct  8 13:41 descriptions.en.tsv.gz
-rw-r--r--   1 pedroszekely  staff  1260472 Oct  8 13:43 item.property.count.compact.tsv.gz
-rw-r--r--   1 pedroszekely  staff   886001 Oct  8 13:41 labels.en.tsv.gz
-rw-r--r--   1 pedroszekely  staff   274394 Oct  8 13:43 metadata.in_degree.tsv.gz
-rw-r--r--   1 pedroszekely  staff   496777 Oct  8 13:42 metadata.out_degree.tsv.gz
-rw-r--r--   1 pedroszekely  staff  1606082 Oct  8 13:24 metadata.pagerank.directed.ordinal.tsv.gz
-rw-r

## Enhance pagerank files to include ordinal

Approach:
- Load the `directed_pagerank` from the metadata file into a dataframe (using kypher because somehow cat is broken, sigh
- Sort the file by pagerank descending
- Add a new column with header `P1545` (ordinal) and store the ranks in this column
- Store the result in a temporary file.

In [12]:
%%time
directed_pagerank = kgtk("""
    query -i $OUT/useful_files/metadata.pagerank.directed.tsv.gz 
    --match '(n1)-[l:Pdirected_pagerank]->(pagerank)'
""")

directed_pagerank_sorted = directed_pagerank.sort_values("node2", ascending=False)
directed_pagerank_sorted.insert(0, 'P1545', range(1, 1 + len(directed_pagerank_sorted)))
directed_pagerank_sorted.to_csv(f"{os.environ['TEMP']}/directed-pagerank.ordinal.tsv", index=False, sep='\t')
directed_pagerank_sorted

[2021-10-08 13:43:49 sqlstore]: DROP graph data table graph_19 from /Users/pedroszekely/Downloads/kypher/projects/tutorial-derived-graphs/useful_files/metadata.pagerank.directed.tsv.gz
[2021-10-08 13:43:49 sqlstore]: IMPORT graph directly into table graph_19 from /Users/pedroszekely/Downloads/kypher/projects/tutorial-derived-graphs/useful_files/metadata.pagerank.directed.tsv.gz ...
[2021-10-08 13:43:50 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_19 AS graph_19_c1
     WHERE graph_19_c1."label" = ?
  PARAS: ['Pdirected_pagerank']
---------------------------------------------
[2021-10-08 13:43:50 sqlstore]: CREATE INDEX on table graph_19 column label ...
[2021-10-08 13:43:50 sqlstore]: ANALYZE INDEX on table graph_19 column label ...

CPU times: user 545 ms, sys: 89.7 ms, total: 634 ms
Wall time: 2.33 s


,P1545,node1,label,node2,id
23658,1,Q23958852,Pdirected_pagerank,0.074847,Q23958852-Pdirected_pagerank-70976
37555,2,Q23960977,Pdirected_pagerank,0.034511,Q23960977-Pdirected_pagerank-112667
13394,3,Q35120,Pdirected_pagerank,0.030465,Q35120-Pdirected_pagerank-40184
10098,4,Q151885,Pdirected_pagerank,0.027707,Q151885-Pdirected_pagerank-30296
11016,5,Q7184903,Pdirected_pagerank,0.011759,Q7184903-Pdirected_pagerank-33050
...,...,...,...,...,...
32303,55947,Q19345728,Pdirected_pagerank,0.000003,Q19345728-Pdirected_pagerank-96911
32291,55948,Q193369,Pdirected_pagerank,0.000003,Q193369-Pdirected_pagerank-96875
32281,55949,Q193326,Pdirected_pagerank,0.000003,Q193326-Pdirected_pagerank-96845
32279,55950,Q1932957,Pdirected_pagerank,0.000003,Q1932957-Pdirected_pagerank-96839


The temporary file looks good, next steps:
- `normalize` to put the qualifiers as extra edges so the file has only `node1/label/node2/id`
- `add-ids` as we want all edges to have ids

In [13]:
kgtk("""
    normalize -i "$TEMP"/directed-pagerank.ordinal.tsv
    / add-id --id-style wikidata 
    -o "$OUT"/useful_files/metadata.pagerank.directed.ordinal.tsv.gz
""")

Look at the result to confirm that we are generating the data we want.

In [14]:
kgtk("""
    head -i "$OUT"/useful_files/metadata.pagerank.directed.ordinal.tsv.gz / add-labels
""")

,node1,label,node2,id,node1;label,label;label
0,Q23958852,Pdirected_pagerank,0.074847,Q23958852-Pdirected_pagerank-70976,'variable-order class'@en,NaN
1,Q23958852-Pdirected_pagerank-70976,P1545,1.000000,Q23958852-Pdirected_pagerank-70976-P1545-6b86b2,NaN,'series ordinal'@en
2,Q23960977,Pdirected_pagerank,0.034511,Q23960977-Pdirected_pagerank-112667,'(meta)class'@en,NaN
3,Q23960977-Pdirected_pagerank-112667,P1545,2.000000,Q23960977-Pdirected_pagerank-112667-P1545-d4735e,NaN,'series ordinal'@en
4,Q35120,Pdirected_pagerank,0.030465,Q35120-Pdirected_pagerank-40184,'entity'@en,NaN
5,Q35120-Pdirected_pagerank-40184,P1545,3.000000,Q35120-Pdirected_pagerank-40184-P1545-4e0740,NaN,'series ordinal'@en
6,Q151885,Pdirected_pagerank,0.027707,Q151885-Pdirected_pagerank-30296,'concept'@en,NaN
7,Q151885-Pdirected_pagerank-30296,P1545,4.000000,Q151885-Pdirected_pagerank-30296-P1545-4b2277,NaN,'series ordinal'@en
8,Q7184903,Pdirected_pagerank,0.011759,Q7184903-Pdirected_pagerank-33050,'abstract object'@en,NaN
9,Q7184903-Pdirected_pagerank-33050,P1545,5.000000,Q7184903-Pdirected_pagerank-33050-P1545-ef2d12,NaN,'series ordinal'@en


Repeat the same steps for `undirected_pagerank`

In [15]:
%%time
undirected_pagerank = kgtk("""
    query -i $OUT/useful_files/metadata.pagerank.undirected.tsv.gz 
    --match '(n1)-[l:Pundirected_pagerank]->(pagerank)'
""")

undirected_pagerank = undirected_pagerank.sort_values("node2", ascending=False)
undirected_pagerank.insert(0, 'P1545', range(1, 1 + len(undirected_pagerank)))
undirected_pagerank.to_csv(f"{os.environ['TEMP']}/undirected-pagerank.ordinal.tsv", index=False, sep='\t')
undirected_pagerank

[2021-10-08 13:43:55 sqlstore]: DROP graph data table graph_20 from /Users/pedroszekely/Downloads/kypher/projects/tutorial-derived-graphs/useful_files/metadata.pagerank.undirected.tsv.gz
[2021-10-08 13:43:55 sqlstore]: IMPORT graph directly into table graph_20 from /Users/pedroszekely/Downloads/kypher/projects/tutorial-derived-graphs/useful_files/metadata.pagerank.undirected.tsv.gz ...
[2021-10-08 13:43:56 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_20 AS graph_20_c1
     WHERE graph_20_c1."label" = ?
  PARAS: ['Pundirected_pagerank']
---------------------------------------------
[2021-10-08 13:43:56 sqlstore]: CREATE INDEX on table graph_20 column label ...
[2021-10-08 13:43:56 sqlstore]: ANALYZE INDEX on table graph_20 column label ...

CPU times: user 564 ms, sys: 95.5 ms, total: 659 ms
Wall time: 2.38 s


,P1545,node1,label,node2,id
4526,1,Q5,Pundirected_pagerank,0.018951,Q5-Pundirected_pagerank-13580
165,2,Q30,Pundirected_pagerank,0.014160,Q30-Pundirected_pagerank-497
4525,3,Q6581097,Pundirected_pagerank,0.008022,Q6581097-Pundirected_pagerank-13577
6512,4,Q15221623,Pundirected_pagerank,0.005540,Q15221623-Pundirected_pagerank-19538
1305,5,Q1860,Pundirected_pagerank,0.004913,Q1860-Pundirected_pagerank-3917
...,...,...,...,...,...
54625,55947,Q8033256,Pundirected_pagerank,0.000004,Q8033256-Pundirected_pagerank-163877
28405,55948,Q16892375,Pundirected_pagerank,0.000004,Q16892375-Pundirected_pagerank-85217
39926,55949,Q27890917,Pundirected_pagerank,0.000004,Q27890917-Pundirected_pagerank-119780
31429,55950,Q10876480,Pundirected_pagerank,0.000004,Q10876480-Pundirected_pagerank-94289


In [16]:
kgtk("""
    normalize -i "$TEMP"/undirected-pagerank.ordinal.tsv
    / add-id --id-style wikidata 
    -o "$OUT"/useful_files/metadata.pagerank.undirected.ordinal.tsv.gz
""")

In [17]:
kgtk("""
    head -i "$OUT"/useful_files/metadata.pagerank.undirected.ordinal.tsv.gz / add-labels
""")

,node1,label,node2,id,node1;label,label;label
0,Q5,Pundirected_pagerank,0.018951,Q5-Pundirected_pagerank-13580,'human'@en,NaN
1,Q5-Pundirected_pagerank-13580,P1545,1.000000,Q5-Pundirected_pagerank-13580-P1545-6b86b2,NaN,'series ordinal'@en
2,Q30,Pundirected_pagerank,0.014160,Q30-Pundirected_pagerank-497,'United States of America'@en,NaN
3,Q30-Pundirected_pagerank-497,P1545,2.000000,Q30-Pundirected_pagerank-497-P1545-d4735e,NaN,'series ordinal'@en
4,Q6581097,Pundirected_pagerank,0.008022,Q6581097-Pundirected_pagerank-13577,'male'@en,NaN
5,Q6581097-Pundirected_pagerank-13577,P1545,3.000000,Q6581097-Pundirected_pagerank-13577-P1545-4e0740,NaN,'series ordinal'@en
6,Q15221623,Pundirected_pagerank,0.005540,Q15221623-Pundirected_pagerank-19538,'bilateral relation'@en,NaN
7,Q15221623-Pundirected_pagerank-19538,P1545,4.000000,Q15221623-Pundirected_pagerank-19538-P1545-4b2277,NaN,'series ordinal'@en
8,Q1860,Pundirected_pagerank,0.004913,Q1860-Pundirected_pagerank-3917,'English'@en,NaN
9,Q1860-Pundirected_pagerank-3917,P1545,5.000000,Q1860-Pundirected_pagerank-3917-P1545-ef2d12,NaN,'series ordinal'@en


## Deploy the tutorial files to `$tutorial_files_path`

Define the files we want to have in the tutorial

In [17]:
tutorial_files_parts = [
    "labels.en.tsv.gz",
    "aliases.en.tsv.gz",
    "descriptions.en.tsv.gz",
    "claims.external-id.tsv.gz",
    "claims.monolingualtext.tsv.gz",
    "claims.quantity.tsv.gz",
    "claims.string.tsv.gz",
    "claims.time.tsv.gz",
    "claims.wikibase-item.tsv.gz",
    "claims.wikibase-property.tsv.gz",
    "qualifiers.tsv.gz"
]

tutorial_files_useful = [
    "derived.P279.tsv.gz",
    "derived.P279star.tsv.gz",
    "derived.P31.tsv.gz",
    "metadata.in_degree.tsv.gz",
    "metadata.out_degree.tsv.gz"
]

Deploy the files from the partition and useful notebooks. 

In [18]:
for file in tutorial_files_parts:
    path = "$OUT/parts/" + file
    !cp -p {path} $tutorial_files_path

for file in tutorial_files_useful:
    path = "$OUT/useful_files/" + file
    !cp -p {path} $tutorial_files_path

Overwrite the original pagerank files with the ones that include ordinal

In [39]:
!cp -p $OUT/useful_files/metadata.pagerank.directed.ordinal.tsv.gz $tutorial_files_path/metadata.pagerank.directed.tsv.gz
!cp -p $OUT/useful_files/metadata.pagerank.undirected.ordinal.tsv.gz $tutorial_files_path/metadata.pagerank.undirected.tsv.gz 

Important to deply the custom KGTK properties file. Copy using KGTK to conveniently compress the file.

In [16]:
kgtk("""
    cat 
        -i "$kgtk_path"/kgtk-properties/kgtk.properties.tsv 
        -i "$OUT"/parts/metadata.property.datatypes.tsv.gz
        -o "$tutorial_files_path"/metadata.property.datatypes.tsv.gz
""")

In [41]:
!ls -l "$tutorial_files_path"

total 79304
-rw-r--r--  1 pedroszekely  staff   1166435 Oct  8 13:40 aliases.en.tsv.gz
-rw-r--r--  1 pedroszekely  staff  12290024 Oct  8 13:40 claims.external-id.tsv.gz
-rw-r--r--  1 pedroszekely  staff    928096 Oct  8 13:40 claims.monolingualtext.tsv.gz
-rw-r--r--  1 pedroszekely  staff   1853807 Oct  8 13:40 claims.quantity.tsv.gz
-rw-r--r--  1 pedroszekely  staff    970374 Oct  8 13:41 claims.string.tsv.gz
-rw-r--r--  1 pedroszekely  staff    660098 Oct  8 13:41 claims.time.tsv.gz
-rw-r--r--  1 pedroszekely  staff   5458682 Oct  8 13:41 claims.wikibase-item.tsv.gz
-rw-r--r--  1 pedroszekely  staff     91306 Oct  8 13:41 claims.wikibase-property.tsv.gz
-rw-r--r--  1 pedroszekely  staff    326969 Oct  8 13:42 derived.P279.tsv.gz
-rw-r--r--  1 pedroszekely  staff   2649699 Oct  8 13:42 derived.P279star.tsv.gz
-rw-r--r--  1 pedroszekely  staff   1006728 Oct  8 13:41 derived.P31.tsv.gz
-rw-r--r--  1 pedroszekely  staff   1157497 Oct  8 13:39 descriptions.tsv.gz
-rw-r--r--  1 pedroszeke

Create an `all.tsv.gz` file

In [42]:
%%time
!kgtk cat -i "$tutorial_files_path"/*.tsv.gz -o "$tutorial_files_path"/all.tsv.gz

CPU times: user 250 ms, sys: 85.7 ms, total: 336 ms
Wall time: 21.8 s


Peek at the file

In [43]:
kgtk("""
    head -i "$tutorial_files_path"/all.tsv.gz
""")

,node1,label,node2,id,node2;wikidatatype
0,P10,alias,'gif'@en,P10-alias-en-282226-0,NaN
1,P10,alias,'animation'@en,P10-alias-en-2f86d8-0,NaN
2,P10,alias,'media'@en,P10-alias-en-c1427e-0,NaN
3,P10,alias,'trailer (Commons)'@en,P10-alias-en-c61ab1-0,NaN
4,P1001,alias,'belongs to jurisdiction'@en,P1001-alias-en-0dd7ce-0,NaN
5,P1001,alias,'linked to jurisdiction'@en,P1001-alias-en-106818-0,NaN
6,P1001,alias,'of jurisdiction'@en,P1001-alias-en-7e4abe-0,NaN
7,P1001,alias,'applied to jurisdiction'@en,P1001-alias-en-89ed18-0,NaN
8,P1001,alias,'jurisdiction'@en,P1001-alias-en-a524ab-0,NaN
9,P1001,alias,'valid in jurisdiction'@en,P1001-alias-en-ca2e7c-0,NaN


Run the KGTK validator on the new knowledge graph

In [15]:
%%time
!kgtk validate -i "$tutorial_files_path"/all.tsv.gz \
    --allow-wikidata-lq-strings True \
    --ignore-minimum-year True \
    --ignore-maximum-year True


Data lines read: 2263488
Data lines passed: 2263488
CPU times: user 1.47 s, sys: 458 ms, total: 1.93 s
Wall time: 2min 1s


### Somehow `kgtk cat` is broken, as it returns nothing. 
The command works if I invoke it before running the config cells at the top, but stops working after I run the config cells, very strange.

In [25]:
!zcat < $OUT/useful_files/metadata.pagerank.directed.tsv.gz | head

node1	label	node2	id
P10	vertex_in_degree	0	P10-vertex_in_degree-0
P10	vertex_out_degree	1	P10-vertex_out_degree-1
P10	Pdirected_pagerank	2.7060194813141665e-06	P10-Pdirected_pagerank-2
Q18610173	vertex_in_degree	15	Q18610173-vertex_in_degree-3
Q18610173	vertex_out_degree	1	Q18610173-vertex_out_degree-4
Q18610173	Pdirected_pagerank	1.800179459943962e-05	Q18610173-Pdirected_pagerank-5
P1000	vertex_in_degree	0	P1000-vertex_in_degree-6
P1000	vertex_out_degree	1	P1000-vertex_out_degree-7
P1000	Pdirected_pagerank	2.7060194813141665e-06	P1000-Pdirected_pagerank-8
zcat: error writing to output: Broken pipe


In [26]:
kgtk("""
    cat -i  $OUT/useful_files/metadata.pagerank.directed.tsv.gz 
""")

,node1,label,node2,id


In [27]:
!kgtk cat -i /Users/pedroszekely/Downloads/kypher/projects/tutorial-derived-graphs/useful_files/metadata.pagerank.directed.tsv.gz  | head

^C

Keyboard interrupt in cat -i /Users/pedroszekely/Downloads/kypher/projects/tutorial-derived-graphs/useful_files/metadata.pagerank.directed.tsv.gz.
